**Importing the required Libraries and Datasets**

In [30]:
import numpy as np
from mlxtend.data import loadlocal_mnist

In [31]:
from sklearn.datasets import fetch_openml
from keras.utils.np_utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split
import time

**Defining Sigmoid ,Sigmoid Derivative and Loss functions**

In [32]:
def sigmoid(x):
  return 1/(1+np.exp(-x))
def sigmoid_derivative(x):
  return x*(1-x)

def loss(predicted_output,desired_output):
  return (desired_output-predicted_output)**2/2

**Importing the Required Dataset**

In [33]:
features, labels = fetch_openml('mnist_784', version=1, return_X_y=True)

In [34]:
features = (features/255).astype('float32')

In [35]:
labels = to_categorical(labels)

**Dividing data into train and test datasets**

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.15, random_state=37)


**Building the Neural Network from the scratch**

In [37]:
class NeuralNetwork():
  def __init__(self,inputLayerNeuronNumber,hiddenLayerNeuronNumber,outputLayerNeuronNumber):
    self.inputLayerNeuronNumber=inputLayerNeuronNumber
    self.hiddenLayerNeuronNumber=hiddenLayerNeuronNumber
    self.outputLayerNeuronNumber=outputLayerNeuronNumber

    self.hidden_weights=np.random.randn(self.hiddenLayerNeuronNumber,self.inputLayerNeuronNumber)*np.sqrt(2/inputLayerNeuronNumber)  #generating random hidden weights from input to hidden layer
    self.hidden_bias=np.zeros([self.hiddenLayerNeuronNumber,1])        #zero bias weights for the hidden layer

    self.output_weights=np.random.randn(self.outputLayerNeuronNumber,self.hiddenLayerNeuronNumber)  #generating random output weights from hidden layer to the output layer
    self.output_bias=np.zeros([self.outputLayerNeuronNumber,1])    #zero bias for the output layer
    self.loss=[]
    self.learning_rate=0.1   #learning rate 

  def train(self,input,desired_output):

    self.hidden_layer_in=np.dot(self.hidden_weights,input)+self.hidden_bias     #Forward propagation to calculate output values
    self.hidden_layer_out=sigmoid(self.hidden_layer_in)

    self.output_layer_in=np.dot(self.output_weights,self.hidden_layer_out)+self.output_bias
    self.predicted_output=sigmoid(self.output_layer_in)

    self.output_error=desired_output-self.predicted_output                 #Backward propagation to calculate errors
    self.d_output_error=self.output_error*(sigmoid_derivative(self.predicted_output))

    self.hidden_layer_error=self.d_output_error.T.dot(self.output_weights)
    self.d_hidden_layer_error=self.hidden_layer_error.T*sigmoid_derivative(self.hidden_layer_out)

    self.output_weights+=self.hidden_layer_out.dot(self.d_output_error.T).T*self.learning_rate
    self.output_bias+=np.sum(self.d_output_error,axis=1,keepdims=True)*self.learning_rate

    self.hidden_weights+=input.dot(self.d_hidden_layer_error.T).T*self.learning_rate       #Updating weights 
    self.hidden_bias+=np.sum(self.d_hidden_layer_error,axis=1,keepdims=True)*self.learning_rate
    self.loss.append(loss(desired_output,self.predicted_output))

  def predict(self, inputs):
      hidden_layer_in = np.dot(self.hidden_weights, inputs) + self.hidden_bias   #Forward propagation on test dataset
      hidden_layer_out = sigmoid(hidden_layer_in)
      output_layer_in = np.dot(self.output_weights, hidden_layer_out) + self.output_bias
      predicted_output = sigmoid(output_layer_in)
      return predicted_output





**Running the Neural Network model for the Dataset**

In [38]:
nn=NeuralNetwork(784,350,10)   #creating Neural Network with 784 input nodes ,350 hidden nodes and 10 output nodes
X_train=np.array(X_train)
Y_train=np.array(Y_train)
for i in range(X_train.shape[0]):          #Stochastic weight update
    inputs = np.array(X_train[i, :])
    inputs=inputs.reshape(-1,1)
    desired_output = np.array(Y_train[i, :])
    desired_output=desired_output.reshape(np.shape(desired_output)[0],1)
    
    nn.train(inputs, desired_output)

**Predicting the output for the test dataset and calculating accuracy**

In [40]:
prediction_list = []
prediction_list.append(nn.predict(X_test.T))
prediction_list=np.array(prediction_list)
prediction_list=prediction_list.T
correct_counter = 0
for i in range(len(prediction_list)):
    out_index = np.where(prediction_list[i] == np.amax(prediction_list[i]))[0][0]
    
    if Y_test[i][out_index] == 1:
        correct_counter+=1

accuracy = correct_counter/10500

print("Accuracy is : ",accuracy*100," %")

Accuracy is :  75.81904761904762  %
